<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
The data that is intially inserted into a NN. It is most of the time the first layer of a NN.
### Hidden Layer:
The layer inbetween the input and output layer that does all the computation of the NN. The reason they are hidden is because it cannot be seen externally.
### Output Layer:
The last layer of most NN that outputs the result of a NN. Data is inserted and computed through a hidden layer(s) (if there is one/many) and outputted through a output layer. 
### Neuron:
A neuron in it's simplest form is just a number in the NN. It consits of the inputs, the weights, and the activation function. The neuron can then take the calculated result and insert that as a input value for another neuron in the NN
### Weight:
A weight is one of the parameters for a neuron. It determines the importance of one node to another.
### Activation Function:
The activation function is the last step before passing the values of a node into another node. The activation function determines if a specific node should be fired or not if the input values is relevant to the predicited value
### Node Map:
The diagram of a Neural Network.
### Perceptron:
A perceptron is a NN with 1 input layer and 1 output layer.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

Each node has it's respected inputs, weights, bias , and activation function. When data is passed into a node, it multiply's all the values by the weight of that specific node, adds everything together to create a weighted sum, then it gets applied to a activation function to determine if the weighted sum is greater than the bias. It is then assigned a value inbetween 1 and 0 and that value gets inserted into another node for further evaluation.

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [65]:
import pandas as pd
import numpy as np
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0],
        'b': [1,1,1,1]
       }

df = pd.DataFrame.from_dict(data).astype('int')
correct_outputs = ([0], [0], [1], [1])

In [2]:
df

,x1,x2,y,b
0,0,0,1,1
1,1,0,1,1
2,0,1,1,1
3,1,1,0,1


In [69]:
##### Your Code Here #####
# Activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [124]:
weights = 2 * np.random.random((4,1)) - 1

In [125]:
weights

array([[-0.86771574],
       [ 0.29633881],
       [-0.54457933],
       [ 0.42676257]])

In [126]:
weighted_sum = np.dot(df, weights)

In [127]:
weighted_sum

array([[-0.11781676],
       [-0.9855325 ],
       [ 0.17852205],
       [-0.14461436]])

In [128]:
activated_output = sigmoid(weighted_sum)
activated_output

array([[0.47057983],
       [0.27179539],
       [0.54451236],
       [0.46390928]])

In [129]:
error = correct_outputs - activated_output

In [130]:
error

array([[-0.47057983],
       [-0.27179539],
       [ 0.45548764],
       [ 0.53609072]])

In [131]:
adjustments = error * sigmoid_derivative(weighted_sum)
adjustments

array([[-0.11723765],
       [-0.05379447],
       [ 0.11296943],
       [ 0.1333244 ]])

In [132]:
weights += np.dot(df.T, adjustments)
weights

array([[-0.78818581],
       [ 0.54263264],
       [-0.60264201],
       [ 0.50202428]])

In [133]:
df.T.shape

(4, 4)

In [134]:
adjustments.shape

(4, 1)

In [74]:
for iteration in range(10000):
    
    weighted_sum = np.dot(df, weights)
    
    activated_output = sigmoid(weighted_sum)
    
    error = correct_outputs - activated_output
    
    adjustments = error * sigmoid_derivative(weighted_sum)
    
    weights += np.dot(df.T, adjustments)
    
print("weights after training")
print(weights)

print("Output after training")
print(activated_output)

weights after training
[[-1.16579407]
 [ 9.12614401]
 [-3.00915348]
 [-1.51768277]]
Output after training
[[0.01069966]
 [0.00335959]
 [0.99004088]
 [0.99841027]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [183]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [190]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

transformer = Normalizer().fit(diabetes[feats])

X = transformer.transform(diabetes[feats])

In [191]:
y = diabetes['Outcome'].values

In [192]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [197]:
##### Update this Class #####

class Perceptron:
    
    def __init__(self, niter = 10):
        self.niter = niter
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        sx = sigmoid(x)
        return sx * (1-sx)

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """
        # Randomly Initialize Weights
        self.weight = 2 * np.random.random((8,1)) - 1

        for i in range(self.niter):
            # Weighted sum of inputs / weights
            weighted_sum = np.dot(X, self.weight) + 1
            # Activate!
            activated_output = self.__sigmoid(weighted_sum)
            # Cac error
            error = y - activated_output
            adjustments = error * self.__sigmoid_derivative(weighted_sum)
            # Update the Weights
            self.weight = self.weight + np.dot(X.T, adjustments)

        return self
    
#     def net_input(self, X):
#         return np.dot(X, self.weight[1:]) + self.weight[0]
    
    
    def predict(self, X):
        """Return class label after unit step"""
        weighted_sum = np.dot(X, self.weight) + 1
        activated_output = self.__sigmoid(weighted_sum)
        return np.round(activated_output)

In [198]:
test = Perceptron()

In [199]:
test.fit(X_train, y_train)

In [200]:
test.predict(X_test)

array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.]])

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?